# 25 Computer storage reviews from BestBuy

In [1]:
import pandas as pd
import os

# Directory containing the CSX review files
directory = "./reviews"

# List to store DataFrames from each file
dfs = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(directory, filename))
        # Drop rows with missing values (if any)
        df = df.dropna()
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
else:
    print("No CSV files found in the 'reviews' directory.")

for idx, df in enumerate(dfs, start=1):
    print(f"DataFrame {idx}: Number of rows: {len(df)}")



DataFrame 1: Number of rows: 520
DataFrame 2: Number of rows: 1020
DataFrame 3: Number of rows: 860
DataFrame 4: Number of rows: 700
DataFrame 5: Number of rows: 680
DataFrame 6: Number of rows: 1020
DataFrame 7: Number of rows: 1020
DataFrame 8: Number of rows: 1020
DataFrame 9: Number of rows: 920
DataFrame 10: Number of rows: 1020
DataFrame 11: Number of rows: 1020
DataFrame 12: Number of rows: 1020
DataFrame 13: Number of rows: 1020
DataFrame 14: Number of rows: 900
DataFrame 15: Number of rows: 1020
DataFrame 16: Number of rows: 640
DataFrame 17: Number of rows: 420
DataFrame 18: Number of rows: 820
DataFrame 19: Number of rows: 840
DataFrame 20: Number of rows: 840
DataFrame 21: Number of rows: 1020
DataFrame 22: Number of rows: 1020
DataFrame 23: Number of rows: 1020
DataFrame 24: Number of rows: 1020
DataFrame 25: Number of rows: 620


In [2]:
df.head(5) 

,brand,model,product,date,review
0,WD,WDBAGF0010BBL-WESN,WD - My Passport 1TB External USB Type-C Porta...,"Nov 20, 2020 2:10 AM","Since my MacBook was about to out of space, I ..."
1,WD,WDBAGF0010BBL-WESN,WD - My Passport 1TB External USB Type-C Porta...,"Oct 6, 2023 4:15 PM",Love this product! It is lightening speed and ...
2,WD,WDBAGF0010BBL-WESN,WD - My Passport 1TB External USB Type-C Porta...,"Apr 17, 2024 11:37 PM",Excellent purchase. Lots and lots of memories ...
3,WD,WDBAGF0010BBL-WESN,WD - My Passport 1TB External USB Type-C Porta...,"Jan 5, 2024 6:20 PM",Been using it for several months. No corrupted...
4,WD,WDBAGF0010BBL-WESN,WD - My Passport 1TB External USB Type-C Porta...,"Jan 8, 2024 6:05 PM","easy setup, very easy to use, can't believe ho..."


In [3]:
print(len(df))
print(len(combined_df))


620
22020


In [4]:
df.dtypes

brand      object
model      object
product    object
date       object
review     object
dtype: object

In [5]:
new_df = pd.DataFrame()

In [6]:
# Sentence Splitting
from nltk.tokenize import sent_tokenize
from cleantext import clean

# Initialize lists to store data
product = []
title = []  # If needed
review = []
date = []
review_id = []

# Loop through each row in the DataFrame
for index, row in combined_df.iterrows():
    sentences = sent_tokenize(row['review'])
    for sentence in sentences:
        product.append(row['product'])
        # title.append(row['title'])  # If needed
        review.append(clean(sentence, no_emoji=True))
        date.append(row['date'])
        review_id.append(index)  # Use the index as review_id

# Create a new DataFrame with the collected data
new_df = pd.DataFrame({
    'review_id': review_id,
    'product': product,
    # 'title': title,  # If needed
    'review': review,
    'date': date
})


In [7]:
new_df.head(5) 

,review_id,product,review,date
0,0,CORSAIR - MP600 PRO LPX 2TB Internal SSD PCIe ...,need extra storage for your ps5 look no further.,"Jun 30, 2023 8:26 PM"
1,0,CORSAIR - MP600 PRO LPX 2TB Internal SSD PCIe ...,caught this on sale for $129.,"Jun 30, 2023 8:26 PM"
2,0,CORSAIR - MP600 PRO LPX 2TB Internal SSD PCIe ...,well worth the money.,"Jun 30, 2023 8:26 PM"
3,0,CORSAIR - MP600 PRO LPX 2TB Internal SSD PCIe ...,zero issues.,"Jun 30, 2023 8:26 PM"
4,0,CORSAIR - MP600 PRO LPX 2TB Internal SSD PCIe ...,took about 2 mins to install.,"Jun 30, 2023 8:26 PM"


In [8]:
print(len(new_df))

88644


# Preprocessing

In [9]:
#Text Preprocessing for aspect extraction
import string
import emoji
import re
import nltk
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

# spacy for lemmatization
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

stopwords = nltk.corpus.stopwords.words('english')

def preprocess(text):
    
    #1. Generating the list of words in the tweet (hastags and other punctuations removed)
    text_blob = TextBlob(text)
    text = ' '.join(text_blob.words)
    
    #2. clean the number 
    text = re.sub(r'[0-9]', '', text)
    
    #3. lower the text
    text = text.lower()
    
    #4. conver the emoji to text form
    text = emoji.demojize(text)
    
    #5. remove punctuation 
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    
    #6. tokenize the text
    text = word_tokenize(text)
    
    #7. remove empty token
    text = [t for t in text if len(t) > 0]
    
    #8. remove non-alphabetical token
    text = [t for t in text if t.isalpha()]
    
    #9. replace the negation token
    replacer  = AntonymReplacer()
    text = replacer.replace_negations(text)
    
    #10. remove the stopwords
    text = [i for i in text if i not in stopwords]
    
    #11. stem the text
    #porter_stemmer = PorterStemmer()
    #text = [porter_stemmer.stem(w) for w in text]
    
    #11. lemmatize the text
    text = lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    return text


def lemmatization(sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(sent)) 
    texts_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return texts_out

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word == 'not' and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

In [10]:
print(new_df.iloc[1,3])
print(type(new_df.iloc[1,3]))
print(preprocess( new_df.iloc[1,3]))

Jun 30, 2023 8:26 PM
<class 'str'>
['pm']


In [11]:
#Preprocess the input data
cleaned_text = []
counter = 1

for text in new_df["review"]:
    print(f'Preprocessing row {counter}')
    counter+=1
    cleaned_text.append(preprocess(str(text)))

new_df["Clean"] = cleaned_text
new_df = new_df.dropna()

new_df.to_excel("preprocessed_data.xlsx")
print('Preprocessing done.')

Preprocessing row 1
Preprocessing row 2
Preprocessing row 3
Preprocessing row 4
Preprocessing row 5
Preprocessing row 6
Preprocessing row 7
Preprocessing row 8
Preprocessing row 9
Preprocessing row 10
Preprocessing row 11
Preprocessing row 12
Preprocessing row 13
Preprocessing row 14
Preprocessing row 15
Preprocessing row 16
Preprocessing row 17
Preprocessing row 18
Preprocessing row 19
Preprocessing row 20
Preprocessing row 21
Preprocessing row 22
Preprocessing row 23
Preprocessing row 24
Preprocessing row 25
Preprocessing row 26
Preprocessing row 27
Preprocessing row 28
Preprocessing row 29
Preprocessing row 30
Preprocessing row 31
Preprocessing row 32
Preprocessing row 33
Preprocessing row 34
Preprocessing row 35
Preprocessing row 36
Preprocessing row 37
Preprocessing row 38
Preprocessing row 39
Preprocessing row 40
Preprocessing row 41
Preprocessing row 42
Preprocessing row 43
Preprocessing row 44
Preprocessing row 45
Preprocessing row 46
Preprocessing row 47
Preprocessing row 48
P

In [12]:
print(type(cleaned_text))

<class 'list'>


In [14]:
import pandas as pd
extracted_data = pd.read_excel("preprocessed_data.xlsx")

# Model Training 

In [23]:
#LDA model training

import gensim.corpora as corpora
import gensim.models as models

from ast import literal_eval
from pprint import pprint

data_words = []
for x in extracted_data['Clean']:
    data_words.append(literal_eval(x))

# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# number of topics
num_topics = 10
num_words = 10

# Build LDA model
lda_model1 =models.LdaMulticore(corpus=corpus,
                              id2word=id2word,
                              num_topics=num_topics)



# Save Model

In [24]:
#save LDA Model
import pickle
topics = lda_model1.print_topics(num_topics=num_topics, num_words=num_words)
topic_list = []
for topic in topics:
  topic_list.append(topic[1])

#df['topics1'] = topic_list1
df_topics = pd.DataFrame(topic_list,  columns =['topics'])

df_topics.to_excel("LDA_topics.xlsx")
pickle.dump(lda_model1, open('Trained Model/LDA_model.model', 'wb'))#save lda model
pickle.dump(id2word, open('Trained Model/LDA_model.dict', 'wb'))#save dictionary
pickle.dump(corpus, open('Trained Model/LDA_model.corpus', 'wb'))#save corpus



In [25]:
pprint(lda_model1.print_topics())
doc_lda = lda_model1[corpus]

[(0,
  '0.028*"drive" + 0.020*"storage" + 0.017*"fast" + 0.014*"work" + '
  '0.014*"time" + 0.014*"issue" + 0.013*"get" + 0.013*"game" + 0.012*"ssd" + '
  '0.012*"use"'),
 (1,
  '0.048*"drive" + 0.032*"ssd" + 0.024*"speed" + 0.021*"fast" + 0.016*"hard" + '
  '0.013*"read" + 0.012*"good" + 0.012*"pc" + 0.012*"great" + 0.011*"price"'),
 (2,
  '0.041*"ssd" + 0.029*"drive" + 0.027*"good" + 0.026*"fast" + 0.017*"storage" '
  '+ 0.016*"use" + 0.016*"game" + 0.013*"speed" + 0.013*"upgrade" + '
  '0.012*"work"'),
 (3,
  '0.059*"use" + 0.022*"device" + 0.020*"get" + 0.020*"file" + 0.018*"new" + '
  '0.014*"work" + 0.013*"easy" + 0.012*"second" + 0.012*"backup" + '
  '0.012*"reliable"'),
 (4,
  '0.018*"buy" + 0.016*"good" + 0.014*"install" + 0.014*"western" + '
  '0.013*"easy" + 0.013*"super" + 0.013*"happy" + 0.012*"digital" + '
  '0.012*"drive" + 0.012*"performance"'),
 (5,
  '0.022*"storage" + 0.019*"game" + 0.016*"drive" + 0.013*"load" + '
  '0.013*"performance" + 0.012*"laptop" + 0.012*"wor

In [26]:
import pickle

lda_model_file = 'LDA_model.model' 
loaded_lda_model = pickle.load(open(lda_model_file, 'rb')) #load trained model
loaded_dictionary = pickle.load(open('LDA_model.dict', 'rb')) #load dictionary 
loaded_corpus = pickle.load(open('LDA_model.corpus', 'rb')) # load corpus


In [4]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

# Prepare visualization
vis = gensimvis.prepare(loaded_lda_model, loaded_corpus, loaded_dictionary)


# Visuslisation

In [28]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.015848  0.009680       1        1  13.073947
1      0.021096 -0.012962       2        1  11.972008
9      0.050052  0.065788       3        1  11.098589
2      0.024737  0.029475       4        1  10.662401
8     -0.078473  0.017936       5        1   9.697288
3     -0.130698  0.011341       6        1   9.670321
0      0.033740  0.010966       7        1   8.866301
5      0.053714  0.062960       8        1   8.852204
4      0.067756 -0.101743       9        1   8.490197
6     -0.026076 -0.093440      10        1   7.616744, topic_info=       Term          Freq         Total Category  logprob  loglift
131   great   7259.000000   7259.000000  Default  30.0000  30.0000
91      use  10258.000000  10258.000000  Default  29.0000  29.0000
128     ssd   9028.000000   9028.000000  Default  28.0000  28.0000
112    work   7606.000000   7606.000000  Default  27.0000  27.0000
83    drive  16979.000000  16979.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
92     fast    368.241632   9742.437999  Topic10  -4.7808  -0.7007
32   instal    257.139567   1997.665255  Topic10  -5.1400   0.5247
47   highly    243.292235   1757.250858  Topic10  -5.1953   0.5976
40      old    248.781460   3194.000932  Topic10  -5.1730   0.0224
131   great    245.678102   7259.707960  Topic10  -5.1856  -0.8113

[785 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
38        1  0.067656  able
38        2  0.180664  able
38        3  0.275086  able
38        4  0.023048  able
38        5  0.052043  able
...     ...       ...   ...
335       8  0.033021   yet
335       9  0.348098   yet
335      10  0.129333   yet
1448      1  0.915084  zone
1448      8  0.041595  zone

[3674 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 10, 3, 9, 4, 1, 6, 5, 7])

In [29]:

# Extract topics
topics = loaded_lda_model.show_topics(num_topics=10, num_words=10, formatted=False)

# Create a list to store the topic strings
topic_strings = []

# Format and print the topics
for topic_num, topic_terms in topics:
    topic_str = f"Topic {topic_num}: " + ", ".join([f"{word}" for word, prob in topic_terms])
    topic_strings.append(topic_str)
    print(topic_str)

Topic 0: drive, storage, fast, work, time, issue, get, game, ssd, use
Topic 1: drive, ssd, speed, fast, hard, read, good, pc, great, price
Topic 2: ssd, drive, good, fast, storage, use, game, speed, upgrade, work
Topic 3: use, device, get, file, new, work, easy, second, backup, reliable
Topic 4: buy, good, install, western, easy, super, happy, digital, drive, performance
Topic 5: storage, game, drive, load, performance, laptop, work, take, time, need
Topic 6: drive, easy, use, purchase, speed, install, perfect, get, storage, recommend
Topic 7: drive, ssd, use, fast, easy, need, pc, storage, datum, hard
Topic 8: fast, drive, use, storage, work, speed, laptop, new, recommend, buy
Topic 9: great, work, product, drive, get, good, storage, price, ssd, buy
